# Phase 2: Label data preprocessing
## Overview
In this section I will aquiring and cleaning data on past solar energy generation for each system, this will be the ground truth of my model. I am continuing to use the OpenEI NREL PVDAQ as a source of data.

## Importing needed libraries

In [1]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import time
import os

sites_df = pd.read_csv('../processed_data/initial_system_metadata.csv')
display(sites_df.head())

,system_id,latitude,longitude,elevation_m,azimuth,tilt,dc_capacity_kW
0,4,39.7406,-105.1774,1795.3,180.0,40.0,1.00
1,10,39.7404,-105.1774,1792.8,180.0,40.0,1.12
2,33,39.7404,-105.1772,1794.0,180.0,40.0,2.40
3,34,36.1952,-115.1582,620.0,180.0,11.2,146.64
4,50,39.7420,-105.1727,1994.7,158.0,45.0,6.00


## Initial data inspection
First I'm going to look at how the data I need is stored in the PVDAQ data lake for an example csv file, in this case system 10000 in 2020. The csv files don't have a clear naming convention so I'm querying AWS so I can see the name of the csv file thats in the directory.

In [55]:
website_url = 'https://oedi-data-lake.s3.amazonaws.com/'
file_base_path = 'pvdaq/csv/pvdata/system_id='
bucket_name = 'oedi-data-lake'

# This method creates a s3 client for AWS so I can later make queries
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

year_file_path = file_base_path + '10000/year=2020/'

# Creates a query for all objects that are contained inside of a directory
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=year_file_path)

# Reads the first item from the above quiery as a csv file
year_data = pd.read_csv(website_url + response['Contents'][0]['Key'], low_memory=False)
display(year_data)

,Unnamed: 0,ac_power_inv_16421_daily_max,ac_power_inv_16421_daily_mean,ac_energy_inv_16421_daily_sum
0,2020-01-01,0.444,0.102297,1.378
1,2020-01-02,0.356,0.086134,1.122
2,2020-01-03,0.444,0.144106,1.523
3,2020-01-04,2.052,0.346154,5.732
4,2020-01-05,2.341,0.246794,3.834
...,...,...,...,...
355,2020-12-27,0.153,0.073937,0.439
356,2020-12-28,0.372,0.093062,0.676
357,2020-12-29,0.138,0.062408,0.301
358,2020-12-30,0.306,0.070386,0.424


# Data filtering
Next I'm going to try and gather 5 years of data for each system. The program will look to see if: each system has data, there's 5+ years of data, only 1 csv file per year, and 350+ days of data per year.

In [4]:
print('The original number of systems: ', len(sites_df))
folder_path = '../raw_data/system_generation_data/'

# Iterates through each system
for index, row in sites_df.iterrows():

    # Holds the data for each year for the system
    contents = []

    # The path for the system folder that the files are in
    file_system_path = file_base_path + str(int(row['system_id'])) + '/'
    
    # Gets all the contents that are in the year folder
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=file_system_path, Delimiter='/')

    # Checks if the system has data
    if 'CommonPrefixes' in response:
        
        # Checks if there are atleast 5 years of data for the system
        if len(response['CommonPrefixes']) >= 5:
            # How many years of data good data has been collected
            collected_years = 0
            # Whether 5 years of good data has successfully been acquired
            success = False
            
            # Itterates through each year for the system in reverse
            for i, year in enumerate(reversed(response['CommonPrefixes'])):
                # Gets the file path for that year
                file_year_path = year['Prefix']
                # Gets all the files in that year folder
                response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=file_year_path)
                 
                # Ensures that there's 1 csv file for the year
                if len(response.get('Contents')) == 1:
                        # Reads that csv file
                        year_data = pd.read_csv(website_url + response['Contents'][0]['Key'], low_memory=False)
            
                        # Checks that it has a sufficient amount of data
                        if len(year_data) >= 350:
                            contents.append(year_data)
                            collected_years += 1

                # Sees if we have enough years worth of data and stops collecting data if so
                if collected_years == 5:
                    success = True
                    break

            # Checks if we succeeded in getting 5 years of good data
            if success:
                # Sets the path that all the csv files for the current system will be stored in
                system_folder_path = folder_path + str(int(row['system_id']))
                # Created a folder at that path if it doesn't exist
                if not os.path.exists(system_folder_path):
                    os.makedirs(system_folder_path)
            
                # Adds each csv file for the system to the folder for the system
                for i, content in enumerate(contents):
                    content.to_csv(system_folder_path + '/' + str(i + 1) + '.csv', index=False)
            
            else:
                # We weren't able to get the data we need for this system so it's dropped
                sites_df.drop(index, inplace=True)
    
        # Not enough years of data so we drop this system
        else:
            sites_df.drop(index, inplace=True)

    # No data for this system so we drop it
    else:
        sites_df.drop(index, inplace=True)

    # Prevents spamming the server too much
    time.sleep(0.1)

    # Indicates progress
    if index % 100 == 0:
        print(index)

print('Systems with 5+ years of data', len(sites_df))
# Saves the usable systems so we dont have to run the proccess again
sites_df.to_csv('../processed_data/useable_systems_metadata.csv', index=False)

The original number of systems:  1224
100
200
300
400
500
600
700
800
900
1000
1100
1200
Systems with 5+ years of data 892


## Calculating mean power produced
I'll now be looking at the data we have downloaded and extracting all of the information we need out of it. For my model I'm extimating the performance ratio, the ratio between power capacity and actual generation. Therefore I will need to find the mean power generation across the 5 years for each system.

In [13]:
final_sites_df = pd.read_csv('../processed_data/useable_systems_metadata.csv')

# Creates the column that will hold the mean generation
final_sites_df['mean_power_generation_kW'] = None
j = 0

# Iterates through each system
for index, row in final_sites_df.iterrows():
    site_id = str(int(row['system_id']))
    five_year_power_sum = 0

    # Opens the csv file for each year and sums each year mean power generation
    for i in range(5):
        year_df = pd.read_csv('../raw_data/system_data/' + site_id + '/' + str(i+1) + '.csv')
        five_year_power_sum += year_df.iloc[:, 2].sum() / len(year_df)

    # Calculates the mean for each system and saves it
    system_power_mean = five_year_power_sum / 5
    final_sites_df.iloc[j,7] = system_power_mean
    j += 1

display(final_sites_df.head())

,system_id,latitude,longitude,elevation_m,azimuth,tilt,dc_capacity_kW,mean_power_generation_kW,performance_ratio
0,10000,44.914573,-93.162525,288.791870,180.0,33.0,5.850,0.940219,0.160721
1,10001,39.483937,-76.301594,51.222542,180.0,20.0,3.360,0.920926,0.274085
2,10003,40.346434,-76.423645,145.150772,180.0,35.0,11.040,3.251637,0.294532
3,10005,33.198982,-97.150581,207.639435,180.0,19.0,15.665,3.438072,0.219475
4,10010,45.136089,-88.010933,202.512192,180.0,35.0,12.192,3.844456,0.315326


## Calculating performance ratio
Next I can calculate the performance ratio for each system


In [14]:
final_sites_df['performance_ratio'] = None
i = 0

# Iterates through each system
for index, row in final_sites_df.iterrows():
    # Calculates the performance ratio and saves it
    final_sites_df.iloc[i, 8] = row['mean_power_generation_kW'] / row['dc_capacity_kW']
    i += 1

final_sites_df.to_csv('../processed_data/useable_systems_metadata.csv', index=False)
display(final_sites_df.head())

,system_id,latitude,longitude,elevation_m,azimuth,tilt,dc_capacity_kW,mean_power_generation_kW,performance_ratio
0,10000,44.914573,-93.162525,288.791870,180.0,33.0,5.850,0.940219,0.160721
1,10001,39.483937,-76.301594,51.222542,180.0,20.0,3.360,0.920926,0.274085
2,10003,40.346434,-76.423645,145.150772,180.0,35.0,11.040,3.251637,0.294532
3,10005,33.198982,-97.150581,207.639435,180.0,19.0,15.665,3.438072,0.219475
4,10010,45.136089,-88.010933,202.512192,180.0,35.0,12.192,3.844456,0.315326
